### Spoken Language Processing
В этом задании предлагается обучить классификатор класса возраста по голосу (пример с тем, как это можно сделать для пола см. в семинаре)

Подумайте, как лучше предсказывать возраст (может быть разбить на группы?) и какой лосс использовать

P.S. не забудьте, что если то вы работает в Colab, то вы можете поменять среду выполнения на GPU/TPU!

Вопросы по заданию/материалам: @Nestyme

In [1]:
!pip3 install timit-utils==0.9.0
!pip3 install torchaudio
! wget https://ndownloader.figshare.com/files/10256148 
!unzip -q 10256148

     ---------------------------------------- 1.0/1.0 MB 939.7 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5888 sha256=f5431e6c34f95595b7917bd6e9b9e324abe8de5ffe23209c05c1a6c4b7d9ce28
  Stored in directory: c:\users\mater\appdata\local\pip\cache\wheels\5b\60\87\28af2605138deac93d162904df42b6fdda1dab9b8757c62aa3
Successfully built python-speech-features


"wget" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.
"unzip" не является внутренней или внешней
командой, исполняемой программой или пакетным файлом.


In [1]:
import timit_utils as tu
import os
import librosa
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import Adam, AdamW, SGD
import torch.nn.functional as F

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
 
import IPython
_TIMIT_PATH = 'data/lisa/data/timit/raw/TIMIT'

## Задание 1
Загрузите данные для обучения. Для этого:
1. Скачайте датасет TIMIT (см семинар)
2. Соберите пары "голос"  — "класс возраста" также, как на семинаре собирались пары "голос"  — "пол". Аудиодорожки сконвертируйте в мелспектрограммы при помощи `torchaudio либо` `librosa`

P.S. вы можете использовать свою реализацию, а можете предложенную (см следующие ячейки)

In [2]:
import timit_utils as tu
import os
import librosa
import numpy as np
from tqdm import tqdm
import torch as t


class timit_dataloader:
    def __init__(self, data_path=_TIMIT_PATH, train_mode=True, age_mode=True):
        self.doc_file_path = os.path.join(data_path, 'DOC', 'SPKRINFO.TXT')
        self.corpus = tu.Corpus(data_path)
        with open(self.doc_file_path) as f:
            self.id_age_dict = dict(
                [(tmp.split(' ')[0], 86 - int(tmp.split('  ')[5].split('/')[-1].replace('??', '50'))) \
                 for tmp in f.readlines()[39:]])
        if train_mode:
            self.trainset = self.create_dataset('train', age_mode=age_mode)
            self.validset = self.create_dataset('valid', age_mode=age_mode)
        self.testset = self.create_dataset('test', age_mode=age_mode)

    def return_age(self, id):
        return self.id_age_dict[id]

    def return_data(self):
        return self.trainset, self.validset, self.testset

    def return_test(self):
        return self.testset

    def create_dataset(self, mode, age_mode=False):
        global people
        assert mode in ['train', 'valid', 'test']
        if mode == 'train':
            people = [self.corpus.train.person_by_index(i) for i in range(350)]
        if mode == 'valid':
            people = [self.corpus.train.person_by_index(i) for i in range(350, 400)]
        if mode == 'test':
            people = [self.corpus.test.person_by_index(i) for i in range(150)]
        spectrograms_and_targets = []
        for person in tqdm(people):
            try:
                target = self.return_age(person.name)
                for i in range(len(person.sentences)):
                    spectrograms_and_targets.append(
                        self.preprocess_sample(person.sentence_by_index(i).raw_audio, target, age_mode=True))
            except Exception as e:
                print(person.name, target)

        X, y = map(np.stack, zip(*spectrograms_and_targets))
        X = X.transpose([0, 2, 1])  # to [batch, time, channels]
        return X, y

    @staticmethod
    def spec_to_image(spec, eps=1e-6):
        mean = spec.mean()
        std = spec.std()
        spec_norm = (spec - mean) / (std + eps)
        spec_min, spec_max = spec_norm.min(), spec_norm.max()
        spec_scaled = 255 * (spec_norm - spec_min) / (spec_max - spec_min)
        spec_scaled = spec_scaled.astype(np.uint8)
        return spec_scaled

    @staticmethod
    def clasterize_by_age(age):
        if age < 25:
            return 0
        elif age < 40:
            return 1
        else:
            return 2

    def preprocess_sample(self, amplitudes, target, age_mode=False, sr=16000, max_length=150):
        spectrogram = librosa.feature.melspectrogram(y=amplitudes, sr=sr, n_mels=128, fmin=1, fmax=8192)[:, :max_length]
        spectrogram = np.pad(spectrogram, [[0, 0], [0, max(0, max_length - spectrogram.shape[1])]], mode='constant')
        target = self.clasterize_by_age(target)
        return self.spec_to_image(np.float32(spectrogram)), target

    def preprocess_sample_inference(self, amplitudes, sr=16000, max_length=150, device='cpu'):
        spectrogram = librosa.feature.melspectrogram(y=amplitudes, sr=sr, n_mels=128, fmin=1, fmax=8192)[:, :max_length]
        spectrogram = np.pad(spectrogram, [[0, 0], [0, max(0, max_length - spectrogram.shape[1])]], mode='constant')
        spectrogram = np.array([self.spec_to_image(np.float32(spectrogram))]).transpose([0, 2, 1])

        return t.tensor(spectrogram, dtype=t.float).to(device, non_blocking=True)

In [4]:
_timit_dataloader = timit_dataloader()
train_data, valid_data, test_data = _timit_dataloader.return_data()

100%|██████████| 150/150 [00:09<00:00, 16.63it/s]


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'using {device} mode')

using cuda mode


In [6]:
from torch.utils.data import TensorDataset, DataLoader

In [7]:
train_dataset = TensorDataset(torch.FloatTensor(train_data[0]), torch.LongTensor(train_data[1]))
valid_dataset = TensorDataset(torch.FloatTensor(valid_data[0]), torch.LongTensor(valid_data[1]))
test_dataset = TensorDataset(torch.FloatTensor(test_data[0]), torch.LongTensor(test_data[1]))

In [8]:
BATCH_SIZE = 64

In [9]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

Простая сверточная сеть, ее можно дотюнить или поменять по желанию

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy


class Model(nn.Module):
    def __init__(self, window_sizes=(3, 4, 5), weight=None):
        super(Model, self).__init__()
        self.weight = weight
        self.convs = nn.ModuleList([
            
            nn.Conv2d(1, 128, [window_size, 128], padding=(window_size - 1, 0))
            for window_size in window_sizes
        ])

        self.fc1 = nn.Linear(128 * len(window_sizes), 3)

    def forward(self, x):
        x = torch.unsqueeze(x, 1)  # [B, C, T, E] Add a channel dim.
        xs = []
        for conv in self.convs:
            x2 = F.relu(conv(x))  # [B, F, T, 1]
            x2 = torch.squeeze(x2, -1)  # [B, F, T]
            x2 = F.dropout(x2, p=0.4)
            x2 = F.max_pool1d(x2, x2.size(2))  # [B, F, 1]
            xs.append(x2)
        x = torch.cat(xs, 2)  # [B, F, window]

        # FC
        x = x.view(x.size(0), -1)  # [B, F * window]
        logits = F.dropout(x, p=0.3)
        logits = self.fc1(x)  # [B, class]
        probs = F.log_softmax(logits, dim=1)
        return probs

    def loss(self, probs, targets):
        return nn.NLLLoss(self.weight)(probs, targets)

# Задание 2
1. Обучите свой классификатор категории возраста
2. Попробуйте улучшить результат. Можно попробовать усложнить сетку, подвигать границы категорий, поискать новые данные, что угодно, кроме учиться на тесте :)
3. Какой подход оказался самым эффективным? Как думаете, почему?
4. Как считаете, где можно было бы применить такой классификатор в качестве вспомогательной задачи?


## Оригинальная модель

In [11]:
model = Model()
if device == torch.device('cuda'):
    model.cuda()
else:
    model.cpu()
model.train()

Model(
  (convs): ModuleList(
    (0): Conv2d(1, 128, kernel_size=(3, 128), stride=(1, 1), padding=(2, 0))
    (1): Conv2d(1, 128, kernel_size=(4, 128), stride=(1, 1), padding=(3, 0))
    (2): Conv2d(1, 128, kernel_size=(5, 128), stride=(1, 1), padding=(4, 0))
  )
  (fc1): Linear(in_features=384, out_features=3, bias=True)
)

In [12]:
optimizer = AdamW(model.parameters(), lr=5e-3)

In [13]:
def train(model, iterator, optimizer, clip):
    model.train()
    epoch_loss = 0
    total_y = []
    total_output = []
    for X, y in iterator:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        output = model(X)
        loss = model.loss(output, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)
        optimizer.step()
        total_y.append(y.cpu().detach())
        total_output.append(output.argmax(axis=1).cpu().detach())
        epoch_loss += loss.item()
    return epoch_loss / len(iterator), accuracy_score(torch.cat(total_output), torch.cat(total_y))

def evaluate(model, iterator):
    model.eval()
    epoch_loss = 0
    total_y = []
    total_output = []
    with torch.no_grad():
        for X, y in iterator:
            X, y = X.to(device), y.to(device)
            output = model(X)
            loss = model.loss(output, y)
            total_y.append(y.cpu().detach())
            total_output.append(output.argmax(axis=1).cpu().detach())
            epoch_loss += loss.item()
    return epoch_loss / len(iterator), accuracy_score(torch.cat(total_output), torch.cat(total_y))

In [14]:
N_EPOCHS = 30
CLIP = 0.005

In [15]:
def loop(model, train_iterator, valid_iterator, optimizer, state_dict_name):
    best_valid_loss = float('inf')
    for epoch in tqdm(range(N_EPOCHS)):
        train_loss, train_acc = train(model, train_iterator, optimizer, CLIP)
        valid_loss, valid_acc = evaluate(model, valid_iterator)
        if epoch % 10 == 0:
            print(f'train_loss: {train_loss:.4f}, train_acc: {train_acc:.4f}, valid_loss: {valid_loss:.4f}, valid_acc: {valid_acc:.4f}')
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), state_dict_name)

In [16]:
loop(model, train_dataloader, valid_dataloader, optimizer, 'best-val-model.pt')

  0%|          | 0/30 [00:00<?, ?it/s]C:\Users\mater\anaconda3\envs\pytorch_gpu\lib\site-packages\torch\nn\functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)
  3%|▎         | 1/30 [00:02<00:58,  2.01s/it]

train_loss: 6.4256, train_acc: 0.5651, valid_loss: 1.0716, valid_acc: 0.6700


 37%|███▋      | 11/30 [00:07<00:10,  1.86it/s]

train_loss: 0.7644, train_acc: 0.7277, valid_loss: 1.8668, valid_acc: 0.6720


 70%|███████   | 21/30 [00:12<00:04,  1.93it/s]

train_loss: 0.5126, train_acc: 0.8166, valid_loss: 3.0777, valid_acc: 0.5960


100%|██████████| 30/30 [00:17<00:00,  1.73it/s]


In [17]:
model.load_state_dict(torch.load('best-val-model.pt'))
test_loss_model, test_acc_model = evaluate(model, test_dataloader)
print(f'test_loss_model: {test_loss_model:.4f}, test_acc_model: {test_acc_model:.4f}')

test_loss_model: 1.0681, test_acc_model: 0.6447


## Изменение window

Посмотрим на поведение модели в случае уменьшения и увеличения количества window.

In [21]:
model_23 = Model(window_sizes=(2, 3))
if device == torch.device('cuda'):
    model_23.cuda()
else:
    model_23.cpu()
model_23.train()

Model(
  (convs): ModuleList(
    (0): Conv2d(1, 128, kernel_size=(2, 128), stride=(1, 1), padding=(1, 0))
    (1): Conv2d(1, 128, kernel_size=(3, 128), stride=(1, 1), padding=(2, 0))
  )
  (fc1): Linear(in_features=256, out_features=3, bias=True)
)

In [22]:
optimizer = AdamW(model_23.parameters(), lr=5e-3)
loop(model_23, train_dataloader, valid_dataloader, optimizer, 'best-val-model-23.pt')

  3%|▎         | 1/30 [00:00<00:24,  1.20it/s]

train_loss: 6.0739, train_acc: 0.5691, valid_loss: 0.9350, valid_acc: 0.6680


 37%|███▋      | 11/30 [00:05<00:09,  2.05it/s]

train_loss: 0.7124, train_acc: 0.7331, valid_loss: 1.2220, valid_acc: 0.6560


 70%|███████   | 21/30 [00:10<00:04,  2.14it/s]

train_loss: 0.5629, train_acc: 0.7880, valid_loss: 1.9778, valid_acc: 0.6440


100%|██████████| 30/30 [00:14<00:00,  2.03it/s]


In [23]:
model_23.load_state_dict(torch.load('best-val-model-23.pt'))
test_loss_model_23, test_acc_model_23 = evaluate(model_23, test_dataloader)
print(f'test_loss_model_23: {test_loss_model_23:.4f}, test_acc_model_23: {test_acc_model_23:.4f}')

test_loss_model_23: 0.9763, test_acc_model_23: 0.6453


**Лосс упал, точность выросла. Двигаясь в сторону уменьшения модели, мы видим рост качества предсказаний.**

In [34]:
model_3456 = Model(window_sizes=(3, 4, 5, 6))
if device == torch.device('cuda'):
    model_3456.cuda()
else:
    model_3456.cpu()
model_3456.train()

Model(
  (convs): ModuleList(
    (0): Conv2d(1, 128, kernel_size=(3, 128), stride=(1, 1), padding=(2, 0))
    (1): Conv2d(1, 128, kernel_size=(4, 128), stride=(1, 1), padding=(3, 0))
    (2): Conv2d(1, 128, kernel_size=(5, 128), stride=(1, 1), padding=(4, 0))
    (3): Conv2d(1, 128, kernel_size=(6, 128), stride=(1, 1), padding=(5, 0))
  )
  (fc1): Linear(in_features=512, out_features=3, bias=True)
)

In [35]:
optimizer = AdamW(model_3456.parameters(), lr=5e-3)
loop(model_3456, train_dataloader, valid_dataloader, optimizer, 'best-val-model-3456.pt')

  3%|▎         | 1/30 [00:00<00:26,  1.11it/s]

train_loss: 7.0999, train_acc: 0.5543, valid_loss: 1.2209, valid_acc: 0.5620


 37%|███▋      | 11/30 [00:07<00:11,  1.65it/s]

train_loss: 0.8465, train_acc: 0.7311, valid_loss: 2.1638, valid_acc: 0.6460


 70%|███████   | 21/30 [00:13<00:05,  1.59it/s]

train_loss: 0.5513, train_acc: 0.8166, valid_loss: 3.6887, valid_acc: 0.5720


100%|██████████| 30/30 [00:18<00:00,  1.60it/s]


In [36]:
model_3456.load_state_dict(torch.load('best-val-model-3456.pt'))
test_loss_model_3456, test_acc_model_3456 = evaluate(model_3456, test_dataloader)
print(f'test_loss_model_3456: {test_loss_model_3456:.4f}, test_acc_model_3456: {test_acc_model_3456:.4f}')

test_loss_model_3456: 1.2493, test_acc_model_3456: 0.6213


**Лосс вырос, точность упала. Двигаясь в сторону усложнения модели, мы видим падение качества предсказаний.**

In [27]:
model_2 = Model(window_sizes=(2,))
if device == torch.device('cuda'):
    model_2.cuda()
else:
    model_2.cpu()
model_2.train()

Model(
  (convs): ModuleList(
    (0): Conv2d(1, 128, kernel_size=(2, 128), stride=(1, 1), padding=(1, 0))
  )
  (fc1): Linear(in_features=128, out_features=3, bias=True)
)

In [28]:
optimizer = AdamW(model_2.parameters(), lr=5e-3)
loop(model_2, train_dataloader, valid_dataloader, optimizer, 'best-val-model-2.pt')

  3%|▎         | 1/30 [00:00<00:19,  1.50it/s]

train_loss: 3.2890, train_acc: 0.5517, valid_loss: 1.1407, valid_acc: 0.6840


 37%|███▋      | 11/30 [00:04<00:07,  2.47it/s]

train_loss: 0.7481, train_acc: 0.7163, valid_loss: 1.1408, valid_acc: 0.6760


 70%|███████   | 21/30 [00:08<00:03,  2.40it/s]

train_loss: 0.6123, train_acc: 0.7614, valid_loss: 1.5565, valid_acc: 0.6420


100%|██████████| 30/30 [00:12<00:00,  2.35it/s]


In [29]:
model_2.load_state_dict(torch.load('best-val-model-2.pt'))
test_loss_model_2, test_acc_model_2 = evaluate(model_2, test_dataloader)
print(f'test_loss_model_2: {test_loss_model_2:.4f}, test_acc_model_2: {test_acc_model_2:.4f}')

test_loss_model_2: 0.9009, test_acc_model_2: 0.6807


**Лосс еще сильнее упал, а точность выросла. Несложно заметить, что изначальная модель сильно переобучалась на тренировочные данные. Вы постепенно уменьшили ее размер, что положительно сказалось на качестве.**

## Веса классов

3 используемых класса сильно несбалансированны. 1 класс очень большой, 2 класс сильно меньше, 3 класс еще меньше. Накинем веса на функцию потерь, чтобы модель больше внимания обращала на самый редкий класс.

In [43]:
model_weight = Model(window_sizes=(2,), weight=torch.FloatTensor([1, 4, 5]).to(device))
if device == torch.device('cuda'):
    model_weight.cuda()
else:
    model_weight.cpu()
model_weight.train()

Model(
  (convs): ModuleList(
    (0): Conv2d(1, 128, kernel_size=(2, 128), stride=(1, 1), padding=(1, 0))
  )
  (fc1): Linear(in_features=128, out_features=3, bias=True)
)

In [44]:
optimizer = AdamW(model_weight.parameters(), lr=5e-3)
loop(model_weight, train_dataloader, valid_dataloader, optimizer, 'best-val-model-weight.pt')

  3%|▎         | 1/30 [00:00<00:17,  1.62it/s]

train_loss: 2.9757, train_acc: 0.5814, valid_loss: 0.8919, valid_acc: 0.6660


 37%|███▋      | 11/30 [00:04<00:07,  2.47it/s]

train_loss: 0.5606, train_acc: 0.7309, valid_loss: 1.3156, valid_acc: 0.6480


 70%|███████   | 21/30 [00:08<00:03,  2.35it/s]

train_loss: 0.4477, train_acc: 0.7591, valid_loss: 2.0080, valid_acc: 0.6540


100%|██████████| 30/30 [00:12<00:00,  2.39it/s]


In [45]:
model_weight.load_state_dict(torch.load('best-val-model-weight.pt'))
test_loss_model_weight, test_acc_model_weight = evaluate(model_weight, test_dataloader)
print(f'test_loss_model_weight: {test_loss_model_weight:.4f}, test_acc_model_weight: {test_acc_model_weight:.4f}')

test_loss_model_weight: 0.7910, test_acc_model_weight: 0.6913


**Лосс упал, но здесь он с весами, поэтому сравнивать с предыдущим экспериментом нельзя. А вот точность выросла, т.е. взвешенный лосс дает прирост качества.**

# Вывод

Почти на 5% повышена точность за счет уменьшения модели и добавления взвешенного лосса. Это произшло из-за сильного переобучения исходной модели на тренировочных данных, а также несбалансированности классов.

Такой классификатор возрастных групп можно использовать, например, при ранжировании поисковых голосовых запросов. Поскольку интересы людей разных возрастов различаются, такой классификатор поможет лучше выдавать результаты поиска пользователям.